In [1]:
import time
import os
import pandas as pd
import numpy as np
import plotly.express as px
from statsmodels.graphics.gofplots import qqplot
from sklearn.model_selection import train_test_split
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax, skew, kurtosis
import seaborn as sns
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline

<h2>Data load</h2>

In [2]:
DATA_PATH_TRAIN = r'C:\Users\user\Downloads\EmEl\Kaggle\30-Days-of-ML-Competition\data\train.csv'
DATA_PATH_TEST = r'C:\Users\user\Downloads\EmEl\Kaggle\30-Days-of-ML-Competition\data\test.csv'

X_train = pd.read_csv(r'C:\Users\user\Downloads\EmEl\Kaggle\30-Days-of-ML-Competition\data\X_train.csv')
X_valid = pd.read_csv(r'C:\Users\user\Downloads\EmEl\Kaggle\30-Days-of-ML-Competition\data\X_valid.csv')
y_train = pd.read_csv(r'C:\Users\user\Downloads\EmEl\Kaggle\30-Days-of-ML-Competition\data\y_train.csv')
y_valid = pd.read_csv(r'C:\Users\user\Downloads\EmEl\Kaggle\30-Days-of-ML-Competition\data\y_valid.csv')


In [18]:
print('Shape of X_train is', X_train.shape)
print('Shape of X_valid is', X_valid.shape)
print('Shape of y_train is', y_train.shape)
print('Shape of y_valid is', y_valid.shape)

Shape of X_train is (210000, 70)
Shape of X_valid is (90000, 70)
Shape of y_train is (210000, 1)
Shape of y_valid is (90000, 1)


In [3]:

from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBRegressor

# n_estimators=400, learning_rate=0.03, max_depth=6, 
#                min_child_weight=4, subsample=0.7, 
#                colsample_bytree=0.7
# Define the model
xgb_model = XGBRegressor(tree_method = 'gpu_hist')
pipe = Pipeline([('xgb', xgb_model)])

param_grid = {
    'xgb__nthread':[4],
    'xgb__objective':['reg:linear'],
    'xgb__max_depth': [5],
    'xgb__n_estimators': [565, 570, 575],
    'xgb__learning_rate': [0.05, 0.06, 0.07],
    'xgb__min_child_weight': [4],
    'xgb__silent': [1],
    'xgb__subsample': [0.7],
    'xgb__colsample_bytree': [0.7],
    'xgb__gamma': [0.005, 0.006],
    'xgb__reg_alpha': [0.0005, 0.0006],
}
grid = RandomizedSearchCV(pipe, param_distributions=param_grid, cv=5, n_jobs=4, verbose=3, n_iter=10)

#  xgb__eval_set=[(X_valid, y_valid)], xgb__early_stopping_rounds=5,
grid.fit(X_train, y_train, 
         xgb__eval_metric='rmse', 
         xgb__verbose=True, 
         xgb__eval_set=[(X_valid, y_valid)], 
         xgb__early_stopping_rounds=7)

grid.best_params_



Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed: 75.6min
[Parallel(n_jobs=4)]: Done  25 tasks      | elapsed: 75.6min
[Parallel(n_jobs=4)]: Done  26 tasks      | elapsed: 75.6min
[Parallel(n_jobs=4)]: Done  27 tasks      | elapsed: 75.6min
[Parallel(n_jobs=4)]: Done  28 tasks      | elapsed: 75.6min
[Parallel(n_jobs=4)]: Done  29 tasks      | elapsed: 75.6min
[Parallel(n_jobs=4)]: Done  30 tasks      | elapsed: 75.6min
[Parallel(n_jobs=4)]: Done  31 tasks      | elapsed: 75.6min


KeyboardInterrupt: 

In [ ]:

'''
RESULTS:

validation_0-rmse:0.72529
{'xgb__subsample': 0.7,
 'xgb__silent': 1,
 'xgb__reg_alpha': 0.0005,
 'xgb__objective': 'reg:linear',
 'xgb__nthread': 4,
 'xgb__n_estimators': 570,
 'xgb__min_child_weight': 4,
 'xgb__max_depth': 5,
 'xgb__learning_rate': 0.06,
 'xgb__gamma': 0.005,
 'xgb__colsample_bytree': 0.7}
 
 rmse:0.72545
 {'xgb__subsample': 0.7,
 'xgb__silent': 1,
 'xgb__objective': 'reg:linear',
 'xgb__nthread': 4,
 'xgb__n_estimators': 550,
 'xgb__min_child_weight': 4,
 'xgb__max_depth': 5,
 'xgb__learning_rate': 0.06,
 'xgb__colsample_bytree': 0.7}
  
'''

In [ ]:

# px.imshow(train_y.corr(),
#             template = 'plotly_dark',
#             height = 800, width = 800)
# report: nothing found

In [ ]:
# outliers check
# for var in num_cols:
#     fig = px.box(train, var,
#                 width = 400, height = 250)
#     fig.show()
# report: nothing found

In [44]:
# encode catigorical variebles with dummies
# train = pd.get_dummies(train)
# train.shape

(300000, 70)

In [ ]:
# qqplot(y, line = 's')
# plt.show()

# print('skewness: %.3f' % (skew(y)))
# print('kurtosis: %.3f' % (kurtosis(y)))

# report: skewness: 0.176
#         kurtosis: 0.713
# which is ok

In [ ]:
x = train.loc[train.index]
y = y.loc[train.index]

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBRegressor

# n_estimators=400, learning_rate=0.03, max_depth=6, 
#                min_child_weight=4, subsample=0.7, 
#                colsample_bytree=0.7
# Define the model
xgb_model = XGBRegressor(random_state=23)
pipe = Pipeline([('xgb', xgb_model)])

param_grid = {
    'xgb__n_estimators': [i for i in range(400,650,50)],
    'xgb__learning_rate': [0.01, 0.03, 0.05],
}
grid = RandomizedSearchCV(pipe, param_distributions=param_grid, cv=4, n_jobs=4, verbose=3)
#  xgb__eval_set=[(X_valid, y_valid)], xgb__early_stopping_rounds=5,
grid.fit(x, y, xgb__verbose=True)

# result: grid.best_params_ = {'xgb__n_estimators': 550, 'xgb__learning_rate': 0.05}

In [58]:
grid.best_params_

{'xgb__n_estimators': 550, 'xgb__learning_rate': 0.05}